In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils import data

import numpy as np

import os
import sys

import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))
from LoadRealRunningJumping import *
from LoadSyntheticRunningJumping import *

from GANModels import *

from dataLoader import *

import torch.fft as fft

from EvaluationFunctions import DagharUniclassEvaluation

from sklearn.manifold import TSNE

from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:55: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [2]:
my_data = np.genfromtxt('../DAGHAR_split_25_10/train/data/KuHar_DAGHAR_Multiclass.csv', delimiter=',')
reshaped_data = my_data.reshape(-1, 6, 30)
print(reshaped_data.shape)
my_data[0:2], reshaped_data[0][0]

(2784, 6, 30)


(array([[-6.57954440e-02,  3.39156203e-03, -3.63780037e-02,
          2.31150724e-03,  1.73688936e-03, -5.11790160e-04],
        [-5.77215701e-02,  2.36059283e-03, -6.14853799e-02,
         -1.77596102e-03,  9.05988214e-04, -4.93116095e-05]]),
 array([-6.57954440e-02,  3.39156203e-03, -3.63780037e-02,  2.31150724e-03,
         1.73688936e-03, -5.11790160e-04, -5.77215701e-02,  2.36059283e-03,
        -6.14853799e-02, -1.77596102e-03,  9.05988214e-04, -4.93116095e-05,
        -4.07599211e-02,  2.29385458e-02, -5.73152825e-02,  7.15994451e-04,
         1.02047459e-03, -1.13943301e-03, -5.91965467e-02,  5.27873542e-03,
        -5.57512864e-02, -3.72491637e-03,  4.45453450e-04,  2.60449527e-03,
        -3.34645994e-02, -8.14894401e-03, -3.63945812e-02,  1.26490995e-04,
        -2.29287613e-03, -3.10051045e-03]))

In [3]:
data_path = '../DAGHAR_split_25_10/train/data/'
my_data = np.genfromtxt('../DAGHAR_split_25_10/train/data/KuHar_DAGHAR_Multiclass.csv', delimiter=',')

new_shape = (my_data.shape[0] // 30, my_data.shape[1], 30)
print(f'new shape: {new_shape}  |  old_shape {my_data.shape}')
reshaped_data = my_data.reshape(new_shape)
r = torch.from_numpy(reshaped_data).unsqueeze(dim=2)
print(r.shape)

dataset = daghar_load_dataset(class_name='KuHar_DAGHAR_Multiclass', path = data_path, seq_len=30)
my_data[:2], dataset[0][0][0],r[0][0][0]

new shape: (2784, 6, 30)  |  old_shape (83520, 6)
torch.Size([2784, 6, 1, 30])
return single class data and labels, class is KuHar_DAGHAR_Multiclass
data shape is (2784, 6, 1, 30)
label shape is (2784,)


(array([[-6.57954440e-02,  3.39156203e-03, -3.63780037e-02,
          2.31150724e-03,  1.73688936e-03, -5.11790160e-04],
        [-5.77215701e-02,  2.36059283e-03, -6.14853799e-02,
         -1.77596102e-03,  9.05988214e-04, -4.93116095e-05]]),
 array([[-0.06579544, -0.05772157, -0.04075992, -0.05919655, -0.0334646 ,
         -0.06759675, -0.08408509, -0.04984349, -0.05933582, -0.0588297 ,
         -0.05476055, -0.05438235, -0.06727322, -0.05350637, -0.05125879,
         -0.05126607, -0.05766883, -0.06653772, -0.05607793, -0.08187018,
         -0.0348706 , -0.05205771, -0.05662755, -0.05324157, -0.05135826,
         -0.06958009, -0.04932166, -0.05904635, -0.06455271, -0.04842236]]),
 tensor([-6.5795e-02,  3.3916e-03, -3.6378e-02,  2.3115e-03,  1.7369e-03,
         -5.1179e-04, -5.7722e-02,  2.3606e-03, -6.1485e-02, -1.7760e-03,
          9.0599e-04, -4.9312e-05, -4.0760e-02,  2.2939e-02, -5.7315e-02,
          7.1599e-04,  1.0205e-03, -1.1394e-03, -5.9197e-02,  5.2787e-03,
         -5.5

In [4]:
data_path = '../DAGHAR_split_25_10/train/label/'
dataset = daghar_load_dataset(class_name='KuHar_Label_Multiclass', path = data_path, seq_len=30)

return single class data and labels, class is KuHar_Label_Multiclass
data shape is (2784, 30, 1)
label shape is (2784,)


In [5]:
my_data = np.genfromtxt('../DAGHAR_split_25_10/train/label/KuHar_Label_Multiclass.csv', delimiter=',',
                       dtype=str)
# Remover aspas de cada elemento e converter para inteiro
my_data = np.char.strip(my_data, '"').astype(int)
print(my_data.shape)
a = my_data.reshape(-1, 30)
a = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=a)
a[464]

(83520,)


1

In [6]:
my_data = np.genfromtxt('../DAGHAR_split_25_10/train/data/KuHar_DAGHAR_Multiclass.csv', delimiter=',')
my_data.shape

(83520, 6)

In [7]:
a = np.array(dataset[:][0])
b = np.array(dataset[:][1])
a.shape, b.shape

((2784, 30, 1), (2784,))

In [8]:
cas = daghar_load_dataset_with_label(class_name = "KuHar_Label_Multiclass", seq_len = 30,
                                     data_path = '../DAGHAR_split_25_10/train/data/KuHar_DAGHAR_Multiclass.csv',
                                     label_path = '../DAGHAR_split_25_10/train/label/KuHar_Label_Multiclass.csv')


(2784, 30, 6)
(2784, 30, 3)
return single class data and labels, class is KuHar_Label_Multiclass
data shape is (2784, 3, 1, 30)
label shape is (2784,)


In [16]:
my_data = np.genfromtxt('../DAGHAR_split_25_10/train/data/KuHar_DAGHAR_Multiclass.csv', delimiter=',')
my_data[:2],cas[0][0][2]

(array([[-6.57954440e-02,  3.39156203e-03, -3.63780037e-02,
          2.31150724e-03,  1.73688936e-03, -5.11790160e-04],
        [-5.77215701e-02,  2.36059283e-03, -6.14853799e-02,
         -1.77596102e-03,  9.05988214e-04, -4.93116095e-05]]),
 array([[-0.036378  , -0.06148538, -0.05731528, -0.05575129, -0.03639458,
         -0.03156457, -0.05273123, -0.07343788, -0.00132329, -0.07544742,
         -0.06331116, -0.02920282, -0.04953403, -0.05010787, -0.07581275,
         -0.06286792, -0.0525714 , -0.06008861, -0.0496536 , -0.08325822,
         -0.08123448, -0.00267412, -0.05966912, -0.07019427, -0.05537245,
         -0.04025771, -0.0348863 , -0.05583232, -0.06486819, -0.03412175]]))

In [10]:
cas[464][0].shape, cas[0:464][1]

((3, 1, 30),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0,